In [1]:
import requests
import json

- Get all shops

In [ ]:
url = "https://www.foody.vn/__get/Place/HomeListPlace?page=1&lat=10.823099&lon=106.629664&count=12&districtId=&cateId=&cuisineId=&isReputation=&type=1"
all_shops_API = requests.get(url, headers={'X-Requested-With': 'XMLHttpRequest',
                                            'Cookie': 'floc=217'})
data = all_shops_API.text
dataJson = json.loads(data)
print(dataJson)

- Get menu of ONE shop: fải loop qua từng request_id trong cell code trên

In [39]:
# VD: có thể thay request_id = 1031704 để thử

request_id = 1031704
delivery_dish_API = requests.get(f"https://gappapi.deliverynow.vn/api/dish/get_delivery_dishes?request_id={request_id}&id_type=1", 
                                    headers={'x-foody-api-version': '1',
                                            'x-foody-app-type': '1004',
                                            'x-foody-client-language': 'vi',
                                            'x-foody-client-type': '1',
                                            'x-foody-client-version': '1',
                                            'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                        }
                                )
data = delivery_dish_API.text
dataJson = json.loads(data)

import pandas as pd
menu_infos = dataJson['reply']['menu_infos']


menu_df = pd.DataFrame(columns=['dish_type_id', 'dish_type_name'])
menu_dish_df = pd.DataFrame(columns=['dish_type_id', 'dish_id', 'dish_name', 'dish_description', 'dish_price_value', 'dish_total_like', 'dish_is_available'])
menu_dish_option_df = pd.DataFrame(columns=['dish_type_id', 'dish_id', 'option_id', 'option_min_select', 'option_max_select'])
menu_dish_option_item_df = pd.DataFrame(columns=['dish_type_id', 'dish_id', 'option_id', 'item_id', 'item_name', 'item_price'])

for menu_info in menu_infos:
        menu_df.loc[len(menu_df.index)] = [menu_info['dish_type_id'], menu_info['dish_type_name']]
        for dish in menu_info['dishes']:
                menu_dish_df.loc[len(menu_dish_df.index)] = [
                        menu_info['dish_type_id'],
                        dish['id'],
                        dish['name'],
                        dish['description'],
                        dish['price']['value'],
                        dish['total_like'],
                        dish['is_available']
                ]
                for option in dish['options']:
                        menu_dish_option_df.loc[len(menu_dish_option_df.index)] = [
                                menu_info['dish_type_id'],
                                dish['id'],
                                option['id'],
                                option['option_items']['min_select'],
                                option['option_items']['max_select']
                        ]
                        for item in option['option_items']['items']:
                                menu_dish_option_item_df.loc[len(menu_dish_option_item_df.index)] = [
                                        menu_info['dish_type_id'],
                                        dish['id'],
                                        option['id'],
                                        item['id'],
                                        item['name'],
                                        item['price']['value']
                                ]
                                
                        
menu_df.to_csv('menu.csv', sep=',', encoding='utf-8')
menu_dish_df.to_csv('menu.dish.csv', sep=',', encoding='utf-8')
menu_dish_option_df.to_csv('menu.dish.option.csv', sep=',', encoding='utf-8')
menu_dish_option_item_df.to_csv('menu.dish.option.item.csv', sep=',', encoding='utf-8')

- get detail of ONE shop

In [46]:
request_id = 1077197
delivery_dish_API = requests.get("https://gappapi.deliverynow.vn/api/delivery/get_detail?request_id={request_id}&id_type=1", 
                                    headers={'x-foody-api-version': '1',
                                            'x-foody-app-type': '1004',
                                            'x-foody-client-language': 'vi',
                                            'x-foody-client-type': '1',
                                            'x-foody-client-version': '1',
                                            'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                        }
                                )
data = delivery_dish_API.text
dataJson = json.loads(data)
print(dataJson)

{'reply': None, 'result': 'error_params'}
